In [1]:
import nltk
import numpy as np
import random
import string
import pyttsx3

# Definición del corpus

In [2]:
f = open(r'C:\Users\ALCEA\1_DTSC\_Proyectos\ASISTENTE VIRTUAL EMMA\Corpus juguetería.txt','r',errors = 'ignore')
raw = f.read()

# Preprocesamiento del texto con NTLK CORPUS

In [3]:
raw = raw.lower() # Conversión en minúscula
sent_tokens = nltk.sent_tokenize(raw) # Conversión del corpus en una lista de sentencias
word_tokens = nltk.word_tokenize(raw) # Conversión del corpus en una lista de palabras
lemmer = nltk.stem.WordNetLemmatizer()

# WordNet: diccionario semántico incluido en NLTK
def LemTokens(tokens): # Función para lematizar cada token
    return [lemmer.lemmatize(token) for token in tokens]

remove_punct_dict = dict((ord(punct), None) for punct in string.punctuation) # Variable para la eliminación de los signos de puntuación
def LemNormalize(text): # Función para eliminar los signos de puntuación del texto en minúscula y tokenizarlo 
    return LemTokens(nltk.word_tokenize(text.lower().translate(remove_punct_dict)))

# Preprocesamiento del texto y evaluación de la similitud entre el mensaje del usuario y el corpus

In [4]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from nltk.corpus import stopwords

# Función para determinar la similitud del texto insertado y el corpus
def response(user_response):
    robo_response = ''
    sent_tokens.append(user_response) # Añade la respuesta de usuario al final del corpus (para poder evaluar la similitud) 
    TfidfVec = TfidfVectorizer(tokenizer = LemNormalize, stop_words = stopwords.words('spanish')) # Convierte las frases en vector
    tfidf = TfidfVec.fit_transform(sent_tokens)
    # Evaluación de la similitud de coseno entre el mensaje del usuario (tfidf[-1]) y el corpus (tfidf)
    vals = cosine_similarity(tfidf[-1], tfidf)
    idx = vals.argsort()[0][-2]
    flat = vals.flatten()
    flat.sort()
    req_tfidf = flat[-2] # Devuelve el vector del corpus que más se aproxima al mensaje del usuario
    
    if(req_tfidf==0): # Ningún vector del corpus se aproxima al mensaje del usuario
        robo_response = robo_response+"Lo siento, no te he entendido. Si no puedo responder a lo que necesitas, ponte en contacto con soporte@juguetesb&b.es"
        return robo_response

    else:
        robo_response = robo_response+sent_tokens[idx]
        return robo_response

# Definición de coincidencias manual

In [5]:
SALUDOS_INPUTS = ["hola", "buenas", "saludos", "qué tal", "hey", "buenos días", "¡hola!", "hola!"]
SALUDOS_OUTPUTS = ["Hola, ¿en qué puedo ayudarte?"]

def saludos(sentence):
    for word in sentence.split():
        if word.lower() in SALUDOS_INPUTS:
            return random.choice(SALUDOS_OUTPUTS)

# Definición del sintetizador de voz

In [6]:
engine = pyttsx3.init()
engine.setProperty('rate', 150)    # Aquí se puede seleccionar la velocidad de la voz
engine.setProperty('voice', 'spanish')
def habla(texto):
    engine.say(texto)
    engine.runAndWait()

# Generación de respuesta

In [7]:
flag = True
print("EMMA: Mi nombre es Emma y contestaré encantada a todas tus preguntas acerca del sitio web de JUGUETES B & B. Si quieres salir, escribe 'Salir' ")
habla("Mi nombre es Emma y contestaré encantada a todas tus preguntas acerca del sitio web de JUGUETES B & B. Si quieres salir, escribe 'Salir' ")
while(flag==True):
    user_response = input()
    user_response = user_response.lower() # Convertimos a minúscula
    
    if(user_response!='salir'):
        
        if(user_response=='gracias' or user_response=='muchas gracias'): # Se podría haber definido otra función de coincidencia manual
            flag=True
            print("EMMA: De nada")
            habla("De nada")
            
        else:
            if(saludos(user_response)!=None): # Si la palabra insertada por el usuario es un saludo (coincidencias manuales definidas previamente)
                print("EMMA: "+saludos(user_response))
                habla(saludos(user_response))
                
            else: # Si la palabra insertada no es un saludo --> CORPUS
                print("EMMA: ",end="") 
                print(response(user_response))
                sent_tokens.remove(user_response) # Para eliminar del corpus la respuesta del usuario y volver a evaluar con el corpus limpio
                habla(response(user_response))
                sent_tokens.remove(user_response) 
    else:
        flag=False
        print("EMMA: Espero volver a verte por aquí, ¡hasta pronto!")
        habla("Espero volver a verte por aquí, ¡hasta pronto!")

EMMA: Mi nombre es Emma y contestaré encantada a todas tus preguntas acerca del sitio web de JUGUETES B & B. Si quieres salir, escribe 'Salir' 


 ¡Hola!


EMMA: Hola, ¿en qué puedo ayudarte?


 ¿De qué opciones de envío dispongo?


EMMA: en juguetes b & b dispones de varias opciones de envío de tu pedido: click and collect, entrega económica, entrega estándar, entrega en punto de recogida y entrega exprés.


...

EMMA: Lo siento, no te he entendido. Si no puedo responder a lo que necesitas, ponte en contacto con soporte@juguetesb&b.es


 Muchas gracias


EMMA: De nada


 Salir


EMMA: Espero volver a verte por aquí, ¡hasta pronto!
